In [1]:
# 🧹 메모리 정리 및 환경 설정
import gc
import os
import sys
from dotenv import load_dotenv

def cleanup_memory():
    """메모리 정리 및 가비지 컬렉션"""
    # 기존 C1 retrieval system이 있다면 정리
    if 'c1_retrieval' in globals():
        try:
            c1_retrieval.close()
            del globals()['c1_retrieval']
            print("✅ 기존 C1 검색 시스템 연결 해제")
        except:
            pass
    
    # 기존 C2 retrieval system이 있다면 정리
    if 'c2_retrieval' in globals():
        try:
            c2_retrieval.close()
            del globals()['c2_retrieval']
            print("✅ 기존 C2 검색 시스템 연결 해제")
        except:
            pass

    # 기존 C3 retrieval system이 있다면 정리
    if 'c3_retrieval' in globals():
        try:
            c3_retrieval.close()
            del globals()['c3_retrieval']
            print("✅ 기존 C3 검색 시스템 연결 해제")
        except:
            pass
    
    # 기존 C4 retrieval system이 있다면 정리
    if 'c4_retrieval' in globals():
        try:
            c4_retrieval.close()
            del globals()['c4_retrieval']
            print("✅ 기존 C4 검색 시스템 연결 해제")
        except:
            pass
    
    # 모듈 캐시 정리 (config 모듈 등 새로고침)
    modules_to_reload = ['config', 'c1_retrieval', 'c2_retrieval', 
                        'prompts.c1_prompt', 'prompts.c2_prompt']
    
    for module_name in modules_to_reload:
        if module_name in sys.modules:
            try:
                del sys.modules[module_name]
                print(f"🔄 {module_name} 모듈 캐시 정리")
            except:
                pass
    
    # 가비지 컬렉션 수행
    collected = gc.collect()
    print(f"🧹 가비지 컬렉션 완료: {collected}개 객체 정리")
    
    # 캐시 정보 확인
    try:
        cache_file = "embedding_cache.json"
        if os.path.exists(cache_file):
            size_mb = os.path.getsize(cache_file) / 1024 / 1024
            print(f"📦 임베딩 캐시 크기: {size_mb:.2f} MB")
    except:
        pass
    
    print("✨ 메모리 정리 완료!")

# 환경 변수 로드
load_dotenv()

# 필수 환경 변수 확인
required_vars = ['OPENAI_API_KEY', 'NEO4J_URI', 'NEO4J_USER', 'NEO4J_PASSWORD']
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"❌ 누락된 환경 변수: {missing_vars}")
    print("💡 .env 파일을 확인해주세요")
else:
    print("✅ 모든 환경 변수 확인됨")

cleanup_memory()

✅ 모든 환경 변수 확인됨
🧹 가비지 컬렉션 완료: 0개 객체 정리
📦 임베딩 캐시 크기: 46.81 MB
✨ 메모리 정리 완료!


In [ ]:
# 🚀 검색 시스템 초기화
from c1_retrieval import C1Retrieval
from c2_retrieval import C2Retrieval
from c3_retrieval import C3Retrieval
from c4_retrieval import C4Retrieval
from config import get_config, get_c1_config, get_c2_config, get_c3_config, get_c4_config

# prompt template 임포트
from prompts.c1_prompt import C1_PROMPT_TEMPLATE
from prompts.c2_prompt import C2_PROMPT_TEMPLATE  
from prompts.c3_prompt import C3_PROMPT_TEMPLATE
from prompts.c4_prompt import C4_PROMPT_TEMPLATE

import openai
from IPython.display import display, HTML, Markdown

def format_cocktail_context(results):
    """검색 결과를 LLM이 이해하기 쉬운 형태로 포맷팅"""
    context_lines = []
    for i, cocktail in enumerate(results, 1):
        context_lines.append(f"{i}. {cocktail['name']}")
        context_lines.append(f"   카테고리: {cocktail.get('category', 'N/A')}")
        context_lines.append(f"   글라스: {cocktail.get('glassType', 'N/A')}")
        context_lines.append(f"   알코올: {cocktail.get('alcoholic', 'N/A')}")
        
        # 레시피 재료 정보
        recipe_ingredients = cocktail.get('recipe_ingredients', [])
        if recipe_ingredients:
            context_lines.append(f"   레시피:")
            for ingredient_info in recipe_ingredients:
                measure = ingredient_info.get('measure', 'unknown')
                ingredient = ingredient_info.get('ingredient', 'unknown')
                context_lines.append(f"     - {measure} {ingredient}")
        else:
            ingredients = cocktail.get('ingredients', [])
            context_lines.append(f"   재료: {', '.join(ingredients)}")
        
        # 제조법 (C4용)
        instructions = cocktail.get('instructions', '')
        if instructions:
            context_lines.append(f"   제조법: {instructions}")
            
        # 설명
        description = cocktail.get('description', '')
        if description:
            context_lines.append(f"   설명: {description}")
            
        # 시각적 설명 (C1용)
        image_desc = cocktail.get('imageDescription', '')
        if image_desc:
            context_lines.append(f"   외관: {image_desc}")
            
        context_lines.append("")  # 빈 줄
    
    return "\n".join(context_lines)

def generate_llm_response(question: str, context: str, prompt_template: str):
    """LLM을 사용하여 답변 생성"""
    try:
        # prompt template에 질문과 컨텍스트 삽입
        formatted_prompt = prompt_template.format(question=question, context=context)
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": formatted_prompt}
            ],
            temperature=0.3
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ LLM 응답 생성 중 오류 발생: {e}"

def display_results_with_llm(question: str, results: list, algorithm_type: str):
    """
    검색 결과와 LLM 답변을 모두 표시
    """
    # 알고리즘별 정보
    algorithm_info = {
        'C1': {
            'name': '🎨 C1 색상-재료 기반 검색',
            'template': C1_PROMPT_TEMPLATE,
            'expert': '시각적 칵테일 검색 전문가'
        },
        'C2': {
            'name': '🍷 C2 Glass Type + 재료 매칭',
            'template': C2_PROMPT_TEMPLATE,
            'expert': '글라스 타입과 재료 매칭 전문가'
        },
        'C3': {
            'name': '🔗 C3 Multi-hop 재료 확장 검색',
            'template': C3_PROMPT_TEMPLATE,
            'expert': 'Multi-hop 재료 확장 기반 칵테일 네트워크 전문가'
        },
        'C4': {
            'name': '🎯 C4 재료기반 유사 레시피 칵테일 추천',
            'template': C4_PROMPT_TEMPLATE,
            'expert': '칵테일 유사도 및 대안 추천 전문가'
        }
    }
    
    algo_info = algorithm_info[algorithm_type]
    
    # 검색 결과 표시
    print(f"\n{algo_info['name']}")
    print("=" * 60)
    print(f"📝 질문: {question}")
    print(f"🔍 찾은 칵테일: {len(results)}개")
    print("=" * 60)
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    # 컨텍스트 준비
    context = format_cocktail_context(results)
    
    # LLM 답변 생성
    print(f"\n🤖 {algo_info['expert']} 답변 생성 중...")
    llm_response = generate_llm_response(question, context, algo_info['template'])
    
    # 결과 표시
    display(HTML(f"""
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0; border: 2px solid #dee2e6;">
        <h2 style="color: #495057; margin-top: 0;">🔍 검색된 칵테일 목록</h2>
        <pre style="background-color: #ffffff; padding: 15px; border-radius: 5px; overflow-x: auto; white-space: pre-wrap;">{context}</pre>
    </div>
    """))
    
    display(HTML(f"""
    <div style="background-color: #e8f4fd; padding: 20px; border-radius: 10px; margin: 20px 0; border-left: 5px solid #007bff;">
        <h2 style="color: #0056b3; margin-top: 0;">🤖 {algo_info['expert']} 답변</h2>
        <div style="white-space: pre-wrap; font-family: 'Segoe UI', Arial, sans-serif; line-height: 1.6; color: #212529;">{llm_response}</div>
    </div>
    """))

def display_c1_results(question, results):
    """C1 검색 결과(색상-재료 기반 알고리즘용)"""
    print(f"\n🎨 C1 색상-재료 기반 검색 결과")
    print("=" * 60)
    print(f"📝 질문: {question}")
    print(f"🔍 찾은 칵테일: {len(results)}개")
    print("=" * 60)
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    for i, cocktail in enumerate(results, 1):
        print(f"\n{i}. 🍸 {cocktail['name']}")
        print(f"   📋 카테고리: {cocktail.get('category', 'N/A')}")
        print(f"   🥃 글라스: {cocktail.get('glassType', 'N/A')}")
        print(f"   🍾 알코올: {cocktail.get('alcoholic', 'N/A')}")
        
        # 재료 표시
        ingredients = cocktail.get('ingredients', [])
        print(f"   🥄 재료: {', '.join(ingredients)}")
        
        # 설명 표시
        description = cocktail.get('description', '')
        print(f"   📖 설명: {description}")
        
        # 시각적 설명 표시 (중요!)
        image_desc = cocktail.get('imageDescription', '')
        print(f"   🎨 외관: {image_desc}")
    
    # LLM 답변 생성 및 표시
    display_results_with_llm(question, results, 'C1')

def display_c2_results(question, results):
    """C2 검색 결과 (Glass Type/Category 기반 알고리즘용)"""
    print(f"\n🍷 C2 Glass Type/Category 기반 검색 결과")
    print("=" * 60)
    print(f"📝 질문: {question}")
    print(f"🔍 찾은 칵테일: {len(results)}개")
    print("=" * 60)
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    for i, cocktail in enumerate(results, 1):
        print(f"\n{i}. 🍸 {cocktail['name']}")
        print(f"   📋 카테고리: {cocktail.get('category', 'N/A')}")
        print(f"   🥃 글라스: {cocktail.get('glassType', 'N/A')}")
        print(f"   🍾 알코올: {cocktail.get('alcoholic', 'N/A')}")
        
        # 재료 표시
        ingredients = cocktail.get('ingredients', [])
        print(f"   🥄 재료: {', '.join(ingredients)}")
        
        # 설명 표시
        description = cocktail.get('description', '')
        print(f"   📖 설명: {description}")
        
        # 시각적 설명 표시
        image_desc = cocktail.get('imageDescription', '')
        print(f"   🎨 외관: {image_desc}")
    
    # LLM 답변 생성 및 표시
    display_results_with_llm(question, results, 'C2')

def display_c3_results(question, results):
    """C3 검색 결과 (Multi-hop 재료 확장용)"""
    print(f"\n🔗 C3 Multi-hop 재료 확장 검색 결과")
    print("=" * 60)
    print(f"📝 질문: {question}")
    print(f"🔍 찾은 칵테일: {len(results)}개")
    print("=" * 60)
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    for i, cocktail in enumerate(results, 1):
        print(f"\n{i}. 🍸 {cocktail['name']}")
        print(f"   📋 카테고리: {cocktail.get('category', 'N/A')}")
        print(f"   🥃 글라스: {cocktail.get('glassType', 'N/A')}")
        print(f"   🍾 알코올: {cocktail.get('alcoholic', 'N/A')}")
        
        # 레시피 재료 표시 (측정값 포함) - 전체 레시피 표시
        recipe_ingredients = cocktail.get('recipe_ingredients', [])
        if recipe_ingredients:
            print(f"   🥄 레시피 ({len(recipe_ingredients)}개 재료):")
            for ingredient_info in recipe_ingredients:
                measure = ingredient_info.get('measure', 'unknown')
                ingredient = ingredient_info.get('ingredient', 'unknown')
                print(f"      - {measure} {ingredient}")
        else:
            ingredients = cocktail.get('ingredients', [])
            print(f"   🥄 재료: {', '.join(ingredients)}")
        
        # 설명 표시
        description = cocktail.get('description', '')
        if description:
            print(f"   📖 설명: {description}")
    
    # LLM 답변 생성 및 표시
    display_results_with_llm(question, results, 'C3')

def display_c4_results(question, results):
    """C4 검색 결과 (관계 기반 + 복잡도)"""
    print(f"\n🎯 C4 유사 칵테일, 재료 복잡도 기반 대안 추천")
    print("=" * 60)
    print(f"📝 질문: {question}")
    print(f"🔍 찾은 칵테일: {len(results)}개")
    print("=" * 60)
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    for i, cocktail in enumerate(results, 1):
        print(f"\n{i}. 🍸 {cocktail['name']}")
        print(f"   📋 카테고리: {cocktail.get('category', 'N/A')}")
        print(f"   🥃 글라스: {cocktail.get('glassType', 'N/A')}")
        print(f"   🍾 알코올: {cocktail.get('alcoholic', 'N/A')}")
        
        # 레시피 재료 표시 (측정값 포함) - 전체 레시피 표시
        recipe_ingredients = cocktail.get('recipe_ingredients', [])
        if recipe_ingredients:
            print(f"   🥄 레시피 ({len(recipe_ingredients)}개 재료):")
            for ingredient_info in recipe_ingredients:
                measure = ingredient_info.get('measure', 'unknown')
                ingredient = ingredient_info.get('ingredient', 'unknown')
                print(f"      - {measure} {ingredient}")
        else:
            ingredients = cocktail.get('ingredients', [])
            print(f"   🥄 재료 ({len(ingredients)}개): {', '.join(ingredients)}")
        
        # 제조법 표시 추가
        instructions = cocktail.get('instructions', '')
        if instructions:
            print(f"   🍹 제조법: {instructions}")
        
        # 설명 표시
        description = cocktail.get('description', '')
        if description:
            print(f"   📖 설명: {description}")
    
    # LLM 답변 생성 및 표시
    display_results_with_llm(question, results, 'C4')

try:
    # 공통 설정 정보
    config = get_config()
    print("🔄 칵테일 검색 시스템 초기화 중...")
    print(f"   • 설정 방식: Python 모듈")
    print(f"   • 임베딩 모델: {config['embedding_model']}")
    print(f"   • LLM 모델: {config['model']}")
    
    # C1 검색 시스템 초기화
    print("\n🎨 C1 색상-재료 기반 검색 시스템 초기화...")
    c1_retrieval = C1Retrieval(use_python_config=True)
    c1_config = get_c1_config()
    print("✅ C1 초기화 완료!")
    print(f"   • 초기 Top-K: {c1_config['initial_top_k']}")
    print(f"   • 최종 Top-K: {c1_config['final_top_k']}")
    print(f"   • 유사도 임계값: {c1_config['similarity_threshold']}")
    
    # C2 검색 시스템 초기화
    print("\n🍷 C2 Glass Type + 계층적 재료 매칭 검색 시스템 초기화...")
    c2_retrieval = C2Retrieval(use_python_config=True)
    c2_config = get_c2_config()
    print("✅ C2 초기화 완료!")
    print(f"   • 초기 Top-K: {c2_config['initial_top_k']}")
    print(f"   • 최종 Top-K: {c2_config['final_top_k']}")
    print(f"   • 목표 후보 수: {c2_config['target_candidates']}")
    print(f"   • 최소 후보 임계값: {c2_config['min_candidates_threshold']}")
    
    # C3 검색 시스템 초기화
    print("\n🔗 C3 Multi-hop 재료 확장 검색 시스템 초기화...")
    c3_retrieval = C3Retrieval(use_python_config=True)
    c3_config = get_c3_config()
    print("✅ C3 초기화 완료!")
    print(f"   • 초기 Top-K: {c3_config['initial_top_k']}")
    print(f"   • 최종 Top-K: {c3_config['final_top_k']}")
    print(f"   • 확장 Top-K: {c3_config['expansion_top_k']}")
    print(f"   • 최소 재료 매치: {c3_config['min_ingredient_match']}")
    
    # C4 검색 시스템 초기화
    print("\n🎯 C4 유사 칵테일, 재료 복잡도 기반 대안 추천 시스템 초기화...")
    c4_retrieval = C4Retrieval(use_python_config=True)
    c4_config = get_c4_config()
    print("✅ C4 초기화 완료!")
    print(f"   • 초기 Top-K: {c4_config['initial_top_k']}")
    print(f"   • 최종 Top-K: {c4_config['final_top_k']}")
    print(f"   • 복잡도 허용범위: {c4_config['complexity_tolerance']}")
    print(f"   • 최소 공유 재료: {c4_config['min_shared_ingredients']}")
    
    # 카테고리 목록 확인 (공통)
    categories = c1_retrieval.get_all_categories()
    print(f"\n📂 DB 카테고리 ({len(categories)}개):")
    print(f"   {', '.join(categories)}")
    
    print("\n🎉 모든 검색 시스템 준비 완료!")
    print("\n💡 C1 알고리즘 (색상-재료 기반 + 재료 공유 확장):")
    print("   1️⃣ 키워드 추출 (visual_keywords 포함)")
    print("   2️⃣ 초기 시각 검색 (질문 ↔ imageDescription)")
    print("   3️⃣ 색상-재료 매칭 (최고 유사도 재료 1개 선정)")
    print("   4️⃣ 재료 공유 관계 확장 (해당 재료를 가진 칵테일 2-3개)")
    print("   5️⃣ 최종 시각 랭킹 (전체 후보 imageDescription 정렬)")
    
    print("\n💡 C2 알고리즘 (Glass Type + 재료 매칭):")
    print("   1️⃣ 키워드 추출 (cocktail, ingredients, glassType, category)")
    print("   2️⃣ Glass Type 결정 (명시적 또는 추론)")
    print("   3️⃣ 해당 Glass Type 칵테일들 필터링")
    print("   4️⃣ 재료 매칭 (모든 재료 → 1개씩 제거하며 확장)")
    print("   5️⃣ 레벨별 imageDescription 유사도 정렬")
    print("   6️⃣ 상위 K개 선택")
    
    print("\n💡 C3 알고리즘 (Multi-hop 재료 확장 + 시각 랭킹):")
    print("   1️⃣ 키워드 추출 (재료 + 칵테일 이름)")
    print("   2️⃣ 초기 검색 (이름 유사도 + 재료 매치)")
    print("   3️⃣ Multi-hop 확장 (재료 → 칵테일 → 공통재료 → 새로운 칵테일)")
    print("   4️⃣ 중복 제거 및 imageDescription 유사도 기반 최종 랭킹")
    print("   5️⃣ 상세 레시피 정보 제공")
    
    print("\n💡 C4 알고리즘 (재료기반 유사 레시피 칵테일 추천):")
    print("   1️⃣ 타겟 칵테일 추출 (이름 또는 재료 기반)")
    print("   2️⃣ 복잡도 필터링 (재료 개수 ±2 범위)")
    print("   3️⃣ 공유 재료 개수로 유사도 계산")
    print("   4️⃣ 공유 재료 개수 기준 정렬")
    print("   5️⃣ 상위 K개 선택")
    
    print("\n🤖 LLM 답변 생성 기능이 추가되었습니다!")
    print("   • 각 검색 결과에 대해 전문가 AI가 답변을 생성합니다")
    print("   • 알고리즘별 특화된 prompt template을 사용합니다")
    
except Exception as e:
    print(f"❌ 검색 시스템 초기화 실패: {e}")
    print("\n🔧 해결 방법:")
    print("1. config.py 파일 존재 확인")
    print("2. prompts/ 폴더 및 프롬프트 파일들 존재 확인")
    print("3. 환경 변수(.env) 확인")
    print("4. Neo4j 데이터베이스 실행 상태 확인")

🔄 칵테일 검색 시스템 초기화 중...
   • 설정 방식: Python 모듈
   • 임베딩 모델: text-embedding-3-small
   • LLM 모델: gpt-4o-mini

🎨 C1 색상-재료 기반 검색 시스템 초기화...
✅ C1 초기화 완료!
   • 초기 Top-K: 6
   • 최종 Top-K: 3
   • 유사도 임계값: 0.7

🍷 C2 Glass Type + 계층적 재료 매칭 검색 시스템 초기화...
✅ C2 초기화 완료!
   • 초기 Top-K: 6
   • 최종 Top-K: 3
   • 목표 후보 수: 5
   • 최소 후보 임계값: 2

🔗 C3 Multi-hop 재료 확장 검색 시스템 초기화...
✅ C3 초기화 완료!
   • 초기 Top-K: 6
   • 최종 Top-K: 3
   • 확장 Top-K: 8
   • 최소 재료 매치: 1

🎯 C4 유사 칵테일, 재료 복잡도 기반 대안 추천 시스템 초기화...
✅ C4 초기화 완료!
   • 초기 Top-K: 6
   • 최종 Top-K: 3
   • 복잡도 허용범위: 2
   • 최소 공유 재료: 1

📂 DB 카테고리 (12개):
   soft drink, homemade liqueur, shake, cocktail, other / unknown, coffee / tea, ordinary drink, beer, punch / party drink, cocoa, shot, milk / float / shake

🎉 모든 검색 시스템 준비 완료!

💡 C1 알고리즘 (색상-재료 기반 + 재료 공유 확장):
   1️⃣ 키워드 추출 (visual_keywords 포함)
   2️⃣ 초기 시각 검색 (질문 ↔ imageDescription)
   3️⃣ 색상-재료 매칭 (최고 유사도 재료 1개 선정)
   4️⃣ 재료 공유 관계 확장 (해당 재료를 가진 칵테일 2-3개)
   5️⃣ 최종 시각 랭킹 (전체 후보 imageDescription 정렬)

💡 C2 알고리즘 (Gla

In [3]:
# c1 🧪 테스트 1: 시각적 특성 기반 검색
image_query = ""
text_qurey = "red layered cocktail with elegant appearance"
question = image_query+text_qurey

print(f"🔍 테스트 쿼리: {question}")
results = c1_retrieval.retrieve(question)
display_c1_results(question, results)

🔍 테스트 쿼리: red layered cocktail with elegant appearance
C1 Retrieval (색상-재료 기반): 사용자 질문 - red layered cocktail with elegant appearance
1단계 - 키워드 추출: {'cocktail': [], 'include_ingredients': [], 'exclude_ingredients': [], 'glassType': [], 'category': ['cocktail'], 'visual_keywords': ['red']}
2단계 - 초기 시각 검색: 6개 칵테일 선정
   → 선정된 칵테일: ['jam donut', 'tequila surprise', 'new york sour', 'abc', 'dirty nipple', 'b-52']
3단계 - 색상 기반 재료 매칭 (색상: ['red'])
   → red와 가장 유사한 재료: red wine (유사도: 0.512)
   → red wine 재료로 확장: wine cooler (이미지 유사도: 0.633)
   → red wine 재료로 확장: gluehwein (이미지 유사도: 0.601)
   → red wine 재료로 확장: sweet sangria (이미지 유사도: 0.589)
   → 색상 재료로 확장된 칵테일: 3개
   → 총 확장된 칵테일: 3개

전체 후보 (중복 제거): 9개 칵테일
   → 전체 후보: ['wine cooler', 'abc', 'sweet sangria', 'jam donut', 'new york sour', 'b-52', 'dirty nipple', 'gluehwein', 'tequila surprise']
최종 이미지 유사도 랭킹: 3개 칵테일 선정
   → 최종 선정: ['jam donut', 'tequila surprise', 'new york sour']
4단계 - 최종 시각 랭킹 완료: 3개 결과

🎨 C1 색상-재료 기반 검색 결과
📝 질문: red layered coc

In [4]:
# c1 🧪 테스트 2: 색상과 재료 조합 검색
image_query = ""
text_qurey = "orange cocktail with tropical vibes"
question = image_query+text_qurey

print(f"🔍 테스트 쿼리: {question}")
results = c1_retrieval.retrieve(question)
display_c1_results(question, results)

🔍 테스트 쿼리: orange cocktail with tropical vibes
C1 Retrieval (색상-재료 기반): 사용자 질문 - orange cocktail with tropical vibes
1단계 - 키워드 추출: {'cocktail': [], 'include_ingredients': [], 'exclude_ingredients': [], 'glassType': [], 'category': ['cocktail'], 'visual_keywords': ['orange']}
2단계 - 초기 시각 검색: 6개 칵테일 선정
   → 선정된 칵테일: ['rum punch', 'barracuda', 'planter_ punch', 'orange push-up', 'space odyssey', 'miami vice']
3단계 - 색상 기반 재료 매칭 (색상: ['orange'])
   → orange와 가장 유사한 재료: orange juice (유사도: 0.536)
   → orange juice 재료로 확장: rum runner (이미지 유사도: 0.693)
   → orange juice 재료로 확장: darkwood sling (이미지 유사도: 0.682)
   → orange juice 재료로 확장: orange crush (이미지 유사도: 0.649)
   → 색상 재료로 확장된 칵테일: 3개
   → 총 확장된 칵테일: 3개

전체 후보 (중복 제거): 9개 칵테일
   → 전체 후보: ['barracuda', 'planter_ punch', 'space odyssey', 'orange push-up', 'orange crush', 'rum punch', 'darkwood sling', 'miami vice', 'rum runner']
최종 이미지 유사도 랭킹: 3개 칵테일 선정
   → 최종 선정: ['rum punch', 'barracuda', 'planter_ punch']
4단계 - 최종 시각 랭킹 완료: 3개 결과

🎨 C1 색상-재료

In [5]:
# c2 🧪 테스트 1: 글라스 타입 기반 검색
image_query = "A refreshing white cocktail with mint and lime served in a highball glass."
text_qurey = "Give me one category that you think this cocktail most likely belongs to, and tell me a cocktail in that category that uses similar ingredients."
question = image_query+text_qurey

print(f"🔍 테스트 쿼리: {question}")
results = c2_retrieval.retrieve(question)
display_c2_results(question, results)

🔍 테스트 쿼리: A refreshing white cocktail with mint and lime served in a highball glass.Give me one category that you think this cocktail most likely belongs to, and tell me a cocktail in that category that uses similar ingredients.
C2 Retrieval (Glass Type + 계층적 재료 매칭): 사용자 질문 - A refreshing white cocktail with mint and lime served in a highball glass.Give me one category that you think this cocktail most likely belongs to, and tell me a cocktail in that category that uses similar ingredients.
1단계 - 키워드 추출: {'cocktail': [], 'include_ingredients': ['mint', 'lime'], 'exclude_ingredients': [], 'glassType': ['highball glass'], 'category': ['Cocktail']}
   → 'highball glass' → 'highball glass' (유사도: 1.000)
2단계 - 대상 글라스 타입: highball glass
3단계 - highball glass 글라스 칵테일: 100개
4단계 - 계층적 재료 매칭
   → 계층적 재료 매칭 시작: ['mint', 'lime']
      'mint' → 'mint' (유사도: 1.000)
      'lime' → 'lime' (유사도: 1.000)
   → Level 0 (모든재료): ['mint', 'lime']
      → 새로 발견한 칵테일: 1개
         ['mojito']
   → Level 1 (재료-1개): 

In [6]:
# c3 🧪 테스트 1: Multi-hop 재료 확장 검색

image_query = "A refreshing white cocktail with mint and lime served in a highball glass."
text_query = "Give me one category that you think this cocktail most likely belongs to, and tell me a cocktail in that category that uses similar ingredients."
question = image_query + text_query

print(f"🔍 테스트 쿼리: {question}")
results = c3_retrieval.retrieve(question)
display_c3_results(question, results)

🔍 테스트 쿼리: A refreshing white cocktail with mint and lime served in a highball glass.Give me one category that you think this cocktail most likely belongs to, and tell me a cocktail in that category that uses similar ingredients.
C3 Retrieval (Multi-hop 재료 확장): 사용자 질문 - A refreshing white cocktail with mint and lime served in a highball glass.Give me one category that you think this cocktail most likely belongs to, and tell me a cocktail in that category that uses similar ingredients.
1단계 - 키워드 추출: {'ingredients': ['mint', 'lime'], 'cocktail_names': []}
   → 재료 기반 초기 검색: 6개 칵테일
3-1단계 - 초기 재료 검색: 6개
   → Multi-hop 확장 결과: 8개 칵테일
      상위 5개: ['frozen daiquiri', 'jitterbug', 'sweet sangria', 'fruit cooler', 'long island tea']
3-2단계 - Multi-hop 확장: 8개
4단계 - 후보 정리: 13개 (중복 제거 후)
   → imageDescription 유사도 랭킹:
      1. mojito (유사도: 0.642)
      2. the laverstoke (유사도: 0.636)
      3. frozen mint daiquiri (유사도: 0.597)
      4. gin smash (유사도: 0.573)
      5. long island tea (유사도: 0.565)
      6

In [7]:
# c4 🧪 테스트 1: 유사 칵테일, 재료 복잡도 기반 대안 추천

image_query = ""
text_query = "Manhattan과 유사한 칵테일 추천해줘. 위스키 베이스로 비슷한 복잡도의 레시피를 원해"
question = image_query + text_query

print(f"🔍 테스트 쿼리: {question}")
results = c4_retrieval.retrieve(question)
display_c4_results(question, results)

🔍 테스트 쿼리: Manhattan과 유사한 칵테일 추천해줘. 위스키 베이스로 비슷한 복잡도의 레시피를 원해
C4 Retrieval (관계 기반 + 복잡도): 사용자 질문 - Manhattan과 유사한 칵테일 추천해줘. 위스키 베이스로 비슷한 복잡도의 레시피를 원해
1단계 - 타겟 정보 추출: {'target_cocktail': 'Manhattan', 'ingredients': ['whiskey']}
   → 타겟 칵테일 발견: manhattan
2단계 - 타겟 칵테일 결정: manhattan

🎯 타겟 칵테일 정보:
   • 이름: manhattan
   • 카테고리: cocktail
   • 글라스: cocktail glass
   • 재료 (6개):
     - 3/4 oz sweet vermouth
     - 2 1/2 oz blended bourbon
     - dash angostura bitters
     - 2 or 3 ice
     - 1 maraschino cherry
     - 1 twist of orange peel

   → 관계 기반 검색: 6개 칵테일
3단계 - 관계 기반 검색: 6개 결과
4단계 - 최종 선정: 3개
   1. old fashioned (공유재료: 2개)
   2. french negroni (공유재료: 2개)
   3. martinez 2 (공유재료: 2개)
5단계 - 관계 기반 검색 완료: 3개 결과

🎯 C4 유사 칵테일, 재료 복잡도 기반 대안 추천
📝 질문: Manhattan과 유사한 칵테일 추천해줘. 위스키 베이스로 비슷한 복잡도의 레시피를 원해
🔍 찾은 칵테일: 4개

1. 🍸 manhattan
   📋 카테고리: cocktail
   🥃 글라스: cocktail glass
   🍾 알코올: Alcoholic
   🥄 레시피 (6개 재료):
      - 3/4 oz sweet vermouth
      - 2 1/2 oz blended bourbon
      - dash angostura b